# 영화 평점 게시글을 이용한 정서 분석

### 목차
- 가설
- 연구방법
- 결론

### 1. 가설
- 나이브 베이즈 분류기를 통해 영화 평점 게시글의 정서를 파악할 수 있다.

### 2. 연구방법
- 2.1 데이터 수집
- 2.2 분류기 생성
- 2.3 분류기 평가

#### 2.1 데이터 수집
- Movie Review Data 이용
- Sentiment polarity(positive or negative)를 분류함
- 5331 positive and 5331 negative processed sentences / snippets. Introduced in Pang/Lee ACL 2005. Released July 2005.
- 출처: http://www.cs.cornell.edu/People/pabo/movie-review-data/

#### 2.2 분류기 생성

##### 2.2.1 Load data
- Positive post 5331개와 negative post 5331개를 로드함 (rt-polarity.pos, rt-polarity.neg)
- 최소 2글자 이상
- 소문자로 변환

In [45]:
from numpy import *

def textParse(bigString):    #input is big string, #output is word list
    listOfTokens = bigString.strip().split(' ')
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

def loadDataSet():
    # positive
    fr=open('data/rt-polarity.pos')
    posPosting = [textParse(inst) for inst in fr.readlines()]
    posLen = len(posPosting)
    posLabels = ones(posLen).tolist()
    
    # negative
    fr=open('data/rt-polarity.neg')
    negPosting = [textParse(inst) for inst in fr.readlines()]
    negLen = len(negPosting)
    negLabels = zeros(negLen).tolist()
    
    # merge
    posPosting.extend(negPosting)
    posLabels.extend(negLabels)
   
    return posPosting, posLabels

In [47]:
listOPosts,listClasses = loadDataSet()
print 'length of post list: %d' % len(listOPosts)
print 'first post of post list: %s' % listOPosts[0]

length of post list: 10662
first post of post list: ['the', 'rock', 'destined', 'the', '21st', "century's", 'new', 'conan', 'and', 'that', "he's", 'going', 'make', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', 'jean-claud', 'van', 'damme', 'steven', 'segal']


##### 2.2.2 Naive basis code
- 분류기 생성

In [52]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        #else: print "the word: %s is not in my Vocabulary!" % word
    return returnVec

def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)      #change to ones() 
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)          #change to log()
    p0Vect = log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

####  2.3 분류기 평가
- 전체 post에 대하여 분류 결과 확인
- 최종 오차율 확인

In [53]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
    
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    # training
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    # test
    errorCount = 0
    for testIndex in range(0, len(listOPosts)):
        thisDoc = array(setOfWords2Vec(myVocabList, listOPosts[testIndex]))    
        if classifyNB(thisDoc,p0V,p1V,pAb) != listClasses[testIndex]:
            errorCount += 1
    print 'the error rate is:', float(errorCount)/len(listOPosts)*100

In [54]:
testingNB()

the error rate is: 6.3215156631


### 3. 결론
- 분류기의 오차율은 약 6%로 확인됨
- 가설에 대해 정확도 94%로 기존의 정서 분류기에 비해 양호한 편임
- 한계점
    - 나이브 베이스 분류기의 특성 상, Training data set에 영향을 많이 받음
    - 단어만으로 분류를 하기 때문에, 문맥을 파악하는데 한계를 지님